# Programmazione Concorrente e Parallela

Module: M02056.2  
Course: C02045.2    
Teachers: Tiziano Leidi, Loris Grossi, Diego Frei

## Threads

### Possibilità di utilizzo
#### Estensione della classe Thread

In [1]:
class MyThread extends Thread {
    public void run() {
        System.out.println("Esecuzione Task #1");
    }
}
Thread thread1 = new MyThread();
thread1.start();


Esecuzione Task #1

#### Utilizzo di un Anonymous Class

In [2]:
Thread thread2 = new Thread(new Runnable() {
    public void run() {
        System.out.println("Esecuzione Task #2");
    }
});
thread2.start();

Esecuzione Task #2


#### Utilizzo di una Lambda Expression

In [3]:
Thread thread3 = new Thread(() -> System.out.println("Esecuzione Task #3"));
thread3.start();

Esecuzione Task #3

### Funzionalità dei threads

#### Sleep
Metodi statici che mettono a dormire il thread corrente per l’intervallo di tempo specificato (o più dell’intervallo, a seconda di quando il thread verrà ri-schedulato).

In [9]:
Thread thread = new Thread(() -> System.out.println("Esecuzione Task dopo 200ms"));
thread.sleep(200); // Sleep for 1s
thread.start();

Esecuzione Task dopo 200ms


#### Join

Mette ad aspettare il thread corrente finché il thread specificato non ha terminato la sua esecuzione.

In [19]:
Thread thread1 = new Thread(() -> {
    System.out.println("Task 1");
    try{
        Thread.sleep(100);
    }
    catch(InterruptedException e){
        
    }
});
Thread thread2 = new Thread(() -> {
    System.out.println("Task 2");
    try{
        Thread.sleep(200);
    }
    catch(InterruptedException e){
        
    }
});

thread1.start();
thread2.start();

thread1.join();
thread2.join();
System.out.println("Tutti i task hanno terminato la loro esecuzione");

Task 1
Task 2
Tutti i task hanno terminato la loro esecuzione


### Esempio

In [23]:
final Thread[] threads = new Thread[10];
for (int i = 0; i < threads.length; i++) {
    threads[i] = new Thread(new Runnable() {
    public void run() {
        System.out.println("Thread started");
        try {
            Thread.sleep((long) Math.random() * 1000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        System.out.println("Thread completed");
    }
    });
    threads[i].start();
}
for (int i = 0; i < threads.length; i++) {
    try {
        threads[i].join();
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
}
System.out.println("Thread all completed");

Thread started
Thread started
Thread completed
Thread started
Thread completed
Thread completed
Thread started
Thread completed
Thread started
Thread started
Thread completed
Thread started
Thread started
Thread started
Thread completed
Thread completed
Thread started
Thread completed
Thread completed
Thread completed
Thread all completed


## Thread Safety

Thread-safe: Il programma ha un comportamento corretto quando è eseguito su più thread  
I programmi single-thread non hanno bisogno di essere thread-safe.

### Memoria condivisa
Nel caso in cui una variabile viene contemporaneamente modificata da più thread, parliamo di ["Race Condition"](https://en.wikipedia.org/wiki/Race_condition)

#### Esempio di Race Condition


In [28]:
class FibonacciNumber {
    private int previousValue = 1;
    private int currentValue = 1;
    
    public int getPreviousValue() {
        return previousValue;
    }
    
    public int getCurrentValue() {
        return currentValue;
    }
    
    public void setNewValue(int value) {
        previousValue = currentValue;
        currentValue = value;
    }
}

class FibonacciCalculator implements Runnable {
    private static int count = 2;
    private FibonacciNumber number = null;
    
    public FibonacciCalculator(FibonacciNumber number) {
        this.number = number;
    }
    public void run() {
        count++;
        number.setNewValue(number.getPreviousValue() +
        number.getCurrentValue());
        System.out.println("The " + count + " Fibonacci number is " + number.getCurrentValue());
    }
}

FibonacciNumber number = new FibonacciNumber();
Thread[] threads = new Thread[20];

for (int i = 0; i < threads.length; i++) {
threads[i] = new Thread(new FibonacciCalculator(number));
threads[i].start();
}

for (int i = 0; i < threads.length; i++) {
    try {
        threads[i].join();
    } catch (InterruptedException e){
        e.printStackTrace();
    }
}

The 43 Fibonacci number is 2
The 44 Fibonacci number is 3
The 45 Fibonacci number is 5
The 46 Fibonacci number is 8
The 47 Fibonacci number is 13
The 48 Fibonacci number is 21
The 49 Fibonacci number is 34
The 51 Fibonacci number is 55
The 51 Fibonacci number is 89
The 52 Fibonacci number is 144
The 54 Fibonacci number is 377
The 55 Fibonacci number is 610
The 53 Fibonacci number is 233
The 56 Fibonacci number is 987
The 57 Fibonacci number is 1597
The 58 Fibonacci number is 2584
The 59 Fibonacci number is 4181
The 60 Fibonacci number is 6765
The 61 Fibonacci number is 10946
The 62 Fibonacci number is 17711


### Shared vs Mutable
Per scrivere codice *thread safe* è necessario gestire correttamente l'accesso a variabile ed oggetti. In particolare alle variabili che sono condivise e mutabili.

##### Shared
La variabile è utilizzata da più threads contemporaneamente

##### Mutable
La variabile può cambiare il valore durante il suo ciclo di vita

#### Ragionamento
Dato che ogni Thread ha il suo stack di esecuzione, le variabili sullo stack non causano problemi. Il problema insorge però quando tentiamo di utilizzare delle variabili che stanno sullo heap e nelle variabili statiche in quanto queste possono essere modificate da altri thread.

### Soluzione per la thread safety
Una soluzione per evitare delle race conditions è proteggere gli stati shared e mutable mediante dei *mutex* (mutual exclusion locks). In questo modo si può garantire ad un thread il lock su una regione di memoria protetta. Si ottiene così la *sincronizzazione di threads*.

### Locks
Se una variabile / oggetto è *sharable e mutable* la sincronizzazione è necessaria **ogni volta che si accede alla variabile in lettura o scrittura**.  
Utilizzare un lock solo sulla scrittura *non serve a nulla*.

### Atomicità
Quando un operazione necessita che un valore modificato sia consistente si parla di atomicità, ossia la modifica è inscindibile (atomica).

#### Compound Actions
Sono sequenze di operazioni che devono eseguire in maniera atomica per essere thread-safe


### Intrinsic locks

#### Synchronized
Un metodo synchronized ha quale lock *l'oggetto in cui è contenuto il metodo*, nel caso di metodi statici viene utilizzato *l'oggetto classe* come oggetto per il lock.

In [32]:
class Counter {
    private long count = 0;
    
    public void add(long value) {
        synchronized (this) {
            this.count += value;
        }
    }
    
    public long get(){
        return this.count;
    }
}

class CounterThread extends Thread {
    private Counter counter = null;
    public CounterThread(Counter counter) {
        this.counter = counter;
    }
    
    @Override
    public void run() {
        for (int i = 0; i < 10; i++) {
            counter.add(i);
        }
        System.out.println("Value: " + counter.get());
    }
}

Counter counter = new Counter();
Thread threadA = new CounterThread(counter);
Thread threadB = new CounterThread(counter);
threadA.start();
threadB.start();

Value: 45
Value: 90


### Explicit Locks
Il supporto è simile a "synchronized" ma è esplicito ed è più flessibile. Sono reentrant locks: ossia il medesimo lock può venir acquisito più vole dallo stesso thread.

In [34]:
import java.util.concurrent.locks.Lock;
public class ConcurrentIntegerArray {
    private Lock lock = new ReentrantLock();
    private int[] arr;
    
    public ConcurrentIntegerArray(int size) {
        arr = new int[size];
    }
    
    public void set(int index, int value) {
        lock.lock();
        try {
            arr[index] = value;
        } finally {
            lock.unlock();
        }
    }
    public int get(int index) {
        lock.lock();
        try {
            return arr[index];
        } finally {
            lock.unlock();
        }
    }
}

### RW-locks
Tanti reader che leggono contemporaneamente, appena c'è un write i reader vengono bloccati fino al termine della scrittura.

In [36]:
import java.util.concurrent.locks.ReadWriteLock;
public class MyConcurrentHashMap<K, V> {
    private Map<K, V> hashMap = new HashMap<>();
    private ReadWriteLock lock = new ReentrantReadWriteLock();
    private Lock readLock = lock.readLock();
    private Lock writeLock = lock.writeLock();

    public void put(K key, V value) {
        writeLock.lock();
        try {
            hashMap.put(key, value);
        } finally {
            writeLock.unlock();
        }
    }
    public V remove(K key) {
        writeLock.lock();
        try {
            return hashMap.remove(key);
        } finally {
            writeLock.unlock();
        }
    }
    
    public V get(K key) {
        readLock.lock();
        try {
        return hashMap.get(key);
        } finally {
        readLock.unlock();
        }
    }
    
    public boolean containsKey(K key) {
        readLock.lock();
        try {
            return hashMap.containsKey(key);
        } finally {
            readLock.unlock();
        }
    }   
}